#Preenchimento de máscara com RoBERTa

💻 [acessar este jupyter notebook no colab](https://colab.research.google.com/drive/1oOXyDOLOleNf8GSoEBOeRvsauVSxrdRy?usp=sharing)

BERT é um modelo de linguagem baseado em transformers, desenvolvido pela equipe do Google AI Language. Introduzido em 2018, BERT marcou um grande avanço no campo do processamento de linguagem natural (PLN) ao fornecer representações contextuais bidirecionais, ou seja, o modelo considera tanto o contexto à esquerda quanto à direita de uma palavra para melhor compreensão de seu significado.

* **RoBERTa (Robustly optimized BERT approach)**

`RoBERTa` é uma variante do BERT desenvolvida pela equipe da Facebook AI, introduzida em 2019. Ele segue a mesma arquitetura básica do BERT, mas com várias modificações e melhorias no processo de pré-treinamento, resultando em um desempenho superior em muitas tarefas de PLN.

O exemplo prático deste notebbok será sobre preenchimento de máscara.

##Configurações iniciais

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

In [ ]:
#baixando o modelo
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

* No texto abaixo existe a entidade `< mask >` que será onde o modelo irá preencher. Criar uma palavra que melhor se adeque ao contexto.

In [ ]:
text = "Uma rede neural é um tipo de <mask> que é usado para processamento de linguagem natural."

In [ ]:
#tokenização
inputs = tokenizer(text, return_tensors='pt')
print(inputs)

{'input_ids': tensor([[    0,   791,  1916, 35117, 26739,  7935,  7252,  4767,   139,   263,
         50264,  1192,  7935,   201,  2102,  3840,   609,  5511,   139,   263,
         38954,  1073,   991,  1632,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}


* Desabilitar o gradiente, pois como não é necessário pode ser desativado do treinamento para melhorar a performance do modelo

In [ ]:
#desabilitar o gradiente
with torch.no_grad():
  outputs = model(**inputs)
print(outputs)

MaskedLMOutput(loss=None, logits=tensor([[[33.0944, -3.7791, 17.9557,  ...,  1.6636,  5.2148, 10.5883],
         [ 6.6143, -2.2842, 16.7855,  ..., -1.4340,  1.5761,  5.9353],
         [ 2.6541, -2.8336, 10.2844,  ..., -3.5465, -0.5512,  1.2722],
         ...,
         [-1.0966, -4.5988,  6.0705,  ..., -4.6206, -6.1500,  1.0818],
         [18.8709, -4.4813, 19.1379,  ..., -0.6549,  2.7602,  6.5704],
         [10.9910, -3.7715, 28.3435,  ..., -1.9791, -4.2113,  8.4504]]]), hidden_states=None, attentions=None)


* No modelo RoBERTa são criados tokens especiais e também algumas palavras podem ser reconhecidas como mais de um token, portanto a posição da palavra pode parecer errada, mas está correta

In [ ]:
#identificando a posição do token mascarado (<mask>) no texto
mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
print(mask_token_index)

tensor([10])


In [ ]:
#logits são as probabilidades para a posição, aqui é feito o print deles vetorizados
mask_token_logits = outputs.logits[0, mask_token_index, :]
print(mask_token_logits)

probabilities = torch.softmax(mask_token_logits, dim=-1)
print(probabilities)

tensor([[-4.9231, -3.1930,  2.1637,  ..., -8.5636, -8.0436, -3.1378]])
tensor([[1.6821e-09, 9.4884e-09, 2.0119e-06,  ..., 4.4134e-11, 7.4234e-11,
         1.0028e-08]])


In [ ]:
#imprime os 5 tokens mais prováveis para substituir a máscara
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
print(top_5_tokens)

[2257, 414, 1468, 35277, 12769]


In [ ]:
#imprime as palavras sem a vetorização
predicted_words = [tokenizer.decode([token]).strip() for token in top_5_tokens]
print(predicted_words)

['software', 'data', 'material', 'arte', 'manual']


Portando, o texto:

> "Uma rede neural é um tipo de `< mask >` que é usado para processamento de linguagem natural."

tem as seguintes palavras para substituir a máscara:

In [ ]:
print(f"Palavras preditas para a máscara: {predicted_words}")

Palavras preditas para a máscara: ['software', 'data', 'material', 'arte', 'manual']
